In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading and preprocessing data

- Imports

In [2]:
import os

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

#from skimage.io import imread
#from skimage.transform import resize
from PIL import Image, ImageOps, ImageFilter
#from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *
from keras.preprocessing.image import *

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [3]:
# Number of breeds

breed_list = os.listdir("/content/drive/MyDrive/input/images/Images/")

num_classes = len(breed_list)
print("{} breeds".format(num_classes))

n_total_images = 0
for breed in breed_list:
    n_total_images += len(os.listdir("/content/drive/MyDrive/input/images/Images/{}".format(breed)))
print("{} images".format(n_total_images))

120 breeds
20580 images


In [4]:
# Reducing to 10 breeds

num_classes = len(breed_list[:10])
print("{} breeds".format(num_classes))

n_total_images = 0
for breed in breed_list[:10]:
    n_total_images += len(os.listdir("/content/drive/MyDrive/input/images/Images/{}".format(breed)))
print("{} images".format(n_total_images))

10 breeds
1837 images


In [5]:
# Mapping of labels and numbers
label_maps = {}
label_maps_rev = {}
for i, v in enumerate(breed_list[:10]):
    label_maps.update({v: i})
    label_maps_rev.update({i : v})

In [6]:
img_dir = "/content/drive/MyDrive/input/images/Images/{}/".format(breed_list[0])
def paths_and_labels_raw():
    paths = list()
    labels = list()
    targets = list()
    for breed in breed_list[:10]:
        base_name = "/content/drive/MyDrive/data_raw/{}/".format(breed)
        for img_name in os.listdir(base_name):
            paths.append(base_name + img_name)
            labels.append(breed)
            targets.append(label_maps[breed])
    return paths, labels, targets

paths_raw, labels_raw, targets_raw = paths_and_labels_raw()

assert len(paths_raw) == len(labels_raw)
assert len(paths_raw) == len(targets_raw)

targets_raw = keras.utils.to_categorical(targets_raw, num_classes=num_classes)

In [7]:
breed_list[:10]

['n02085936-Maltese_dog',
 'n02085620-Chihuahua',
 'n02085782-Japanese_spaniel',
 'n02086240-Shih-Tzu',
 'n02086646-Blenheim_spaniel',
 'n02086079-Pekinese',
 'n02087046-toy_terrier',
 'n02086910-papillon',
 'n02087394-Rhodesian_ridgeback',
 'n02089973-English_foxhound']

In [8]:
#train/test split

train_data_dir = "/content/drive/MyDrive/data_raw"
img_width = 224
img_height = 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2) # set validation split

train_generator_raw = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator_raw = train_datagen.flow_from_directory(
    train_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 1474 images belonging to 10 classes.
Found 363 images belonging to 10 classes.


## Inception V3

In [9]:
BATCH_SIZE = 32

In [10]:
from keras.models import Model
from keras.optimizers import adam_v2
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the InceptionV3 model so we can do transfer learning
base_inception = InceptionV3(weights='imagenet', include_top=False, 
                             input_shape=(224, 224, 3))
                             
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
total_classes = targets_raw.shape[1]
predictions = Dense(total_classes, activation='softmax')(out)

model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
model.compile(adam_v2.Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
# Train the model
batch_size = BATCH_SIZE
train_steps_per_epoch = train_generator_raw.samples // batch_size
val_steps_per_epoch = validation_generator_raw.samples // batch_size

history = model.fit_generator(train_generator_raw,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=validation_generator_raw,
                              validation_steps=val_steps_per_epoch,
                              epochs=15, verbose=1)

Epoch 1/15
46/46 [==============================] - 171s 4s/step - loss: 0.6847 - accuracy: 0.8433 - val_loss: 0.1502 - val_accuracy: 0.9659
Epoch 2/15
46/46 [==============================] - 163s 4s/step - loss: 0.1380 - accuracy: 0.9612 - val_loss: 0.1091 - val_accuracy: 0.9744
Epoch 3/15
46/46 [==============================] - 163s 4s/step - loss: 0.1003 - accuracy: 0.9695 - val_loss: 0.1079 - val_accuracy: 0.9659
Epoch 4/15
46/46 [==============================] - 163s 4s/step - loss: 0.0773 - accuracy: 0.9792 - val_loss: 0.1133 - val_accuracy: 0.9631
Epoch 5/15
46/46 [==============================] - 163s 4s/step - loss: 0.0594 - accuracy: 0.9847 - val_loss: 0.1115 - val_accuracy: 0.9631
Epoch 6/15
46/46 [==============================] - 163s 4s/step - loss: 0.0501 - accuracy: 0.9875 - val_loss: 0.1112 - val_accuracy: 0.9602
Epoch 7/15
46/46 [==============================] - 163s 4s/step - loss: 0.0428 - accuracy: 0.9910 - val_loss: 0.0986 - val_accuracy: 0.9716
Epoch 8/15
46

In [12]:
train_data_dir = "/content/drive/MyDrive/data"
img_width = 224
img_height = 224
batch_size = 32

train_datagen_mc = ImageDataGenerator(rescale=1./255,
                                   zca_whitening=True,
                                   horizontal_flip=True, 
                                   validation_split=0.2) # set validation split

train_generator_1 = train_datagen_mc.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator_1 = train_datagen_mc.flow_from_directory(
    train_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 1474 images belonging to 10 classes.
Found 363 images belonging to 10 classes.


In [13]:
# Train the model

train_steps_per_epoch = train_generator_1.samples // batch_size
val_steps_per_epoch = validation_generator_1.samples // batch_size

history = model.fit_generator(train_generator_1,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=validation_generator_1,
                              validation_steps=val_steps_per_epoch,
                              epochs=15, verbose=1)

Epoch 1/15
46/46 [==============================] - 587s 13s/step - loss: 0.3587 - accuracy: 0.8842 - val_loss: 0.4207 - val_accuracy: 0.8494
Epoch 2/15
46/46 [==============================] - 165s 4s/step - loss: 0.2148 - accuracy: 0.9300 - val_loss: 0.3468 - val_accuracy: 0.8835
Epoch 3/15
46/46 [==============================] - 165s 4s/step - loss: 0.1719 - accuracy: 0.9397 - val_loss: 0.4007 - val_accuracy: 0.8722
Epoch 4/15
46/46 [==============================] - 165s 4s/step - loss: 0.1397 - accuracy: 0.9438 - val_loss: 0.3990 - val_accuracy: 0.8665
Epoch 5/15
46/46 [==============================] - 165s 4s/step - loss: 0.1009 - accuracy: 0.9716 - val_loss: 0.4093 - val_accuracy: 0.8835
Epoch 6/15
46/46 [==============================] - 165s 4s/step - loss: 0.0816 - accuracy: 0.9743 - val_loss: 0.4178 - val_accuracy: 0.8835
Epoch 7/15
46/46 [==============================] - 164s 4s/step - loss: 0.0567 - accuracy: 0.9847 - val_loss: 0.4687 - val_accuracy: 0.8722
Epoch 8/15
4